In [42]:
# Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

# Dataset
player_data = pd.read_csv('../data/raw/match_players.csv')

player_data = player_data[['player_id', 'player_nick']].drop_duplicates()

# SAVE THE DATASET
player_data.to_csv('../data/interim/player_data.csv', index=False)

player_data.head()



,player_id,player_nick
0,5736,kNgV-
1,2532,mch
2,7382,steel
3,5698,pava
4,10563,caike


In [43]:
# Input has to be a string array of len 5
def get_team_id_from_name(player_names_list):
    players = player_data[player_data['player_nick'].isin(player_names_list)]

    return players.groupby('player_nick').first()['player_id'].to_list()

get_team_id_from_name([
    'kNgV-',
    'mch',
    'steel',
    'pava',
    'caike']
    )

[10563, 5736, 2532, 5698, 7382]

In [44]:
from pickle import load

with open("../models/scalers/standard_scaler.pkl", 'rb') as file:
    scaler = load(file)

In [69]:
team1 = get_team_id_from_name(['kNgV-', 'mch', 'steel', 'pava', 'caike'])

team2 = get_team_id_from_name(['lrz', 'invul', 'gut0', 'fv', 'cogu'])

# Input params have to be list-like
def transform_input(team1, team2):
    team1 = pd.DataFrame(team1)
    team2 = pd.DataFrame(team2)

    hash_function = lambda x: hash(frozenset(x))
    team1 = team1.apply(hash_function)
    team2 = team2.apply(hash_function)

    transformed_input = scaler.transform(pd.concat([team1,team2], axis=1))

    return [transformed_input, np.array([[0.0 for i in range(15)]])]

transformation_result = transform_input(team1, team2)


/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[array([[0.92070511, 1.71417266]]),
 array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])]

In [70]:
with open("../models/csgo_match_results_predictor-beta1.pkl", 'rb') as file:
    model = load(file)

model.predict(transformation_result)

1/1 [==============================] - 0s 72ms/step


array([[1.]], dtype=float32)

In [75]:
class InputTransformer:
    player_data = pd.read_csv('../data/interim/player_data.csv')

    scaler = scaler

    def __init__(self):
        pass

    def _get_team_id_from_name(self, player_names_list):
        players = player_data[player_data['player_nick'].isin(player_names_list)]
        return players.groupby('player_nick').first()['player_id'].to_list()

    def transform_input(self, team1, team2):
        team1 = self._get_team_id_from_name(team1)
        team2 = self._get_team_id_from_name(team2)

        team1 = pd.DataFrame(team1)
        team2 = pd.DataFrame(team2)

        hash_function = lambda x: hash(frozenset(x))
        team1 = team1.apply(hash_function)
        team2 = team2.apply(hash_function)

        transformed_input = scaler.transform(pd.concat([team1,team2], axis=1))

        return [transformed_input, np.array([[0.0 for i in range(15)]])]

# Create an instance
obj = InputTransformer()

team1 = get_team_id_from_name(['kNgV-', 'mch', 'steel', 'pava', 'caike'])

team2 = get_team_id_from_name(['lrz', 'invul', 'gut0', 'fv', 'cogu'])

transformed_input = obj.transform_input(['kNgV-', 'mch', 'steel', 'pava', 'caike'], ['lrz', 'invul', 'gut0', 'fv', 'cogu'])

model.predict(transformed_input)


1/1 [==============================] - 0s 15ms/step


/home/alamo/.pyenv/versions/3.11.5/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([[1.]], dtype=float32)

In [ ]:
from pickle import dump

with open("../models/input_transformer.pkl", "wb") as f:
    dump(obj, f)